# 导入模块

In [1]:
from operator import index
from tokenize import group

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False

import warnings
warnings.filterwarnings("ignore")

import pymysql
from sqlalchemy import create_engine, column
import json
import datetime
from datetime import timedelta ,time
import re
import os
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean, Risk_Data

In [2]:
pd.set_option('display.max_columns', None)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()
risk_data = Risk_Data()


# 链接数据库

In [3]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 导入数据

In [4]:
sql = '''
select tprm.order_create_time, tprm.rembursement_status, om.merchant_name,tod.sku_attributes,
ymos.order_id, ymos.sort, ymos.reality_refund_date -- , too.pay_date
from db_digua_business.t_postlease_receivables_monitoring tprm
left join db_rent.ya_merchant_order_stages ymos on ymos.order_id=tprm.order_id
left join db_digua_business.t_order om on om.id=tprm.order_id
left join db_digua_business.t_order_details tod on om.id = tod.order_id
-- left join db_digua_business.t_order_out too on too.order_id=tprm.order_id
where date_format(tprm.order_create_time, '%Y-%m')>='2023-01'
-- and too.status=2
and tprm.rembursement_status='已买断'
'''
# rembursement_status 还款状态
df = query(sql)

In [5]:
df = clean.drop_merchant(df)
df = clean.drop_rejected_merchant(df)

In [6]:
sql2 = '''
select order_id, pay_date from db_digua_business.t_order_out
where status=2
'''
df2 = query(sql2)

In [7]:
df.loc[:, '实付日期'] = df.reality_refund_date.dt.date
df.loc[:, '月份'] = df.order_create_time.dt.strftime('%Y-%m')
df2.loc[:, '支付日期'] = df2.pay_date.dt.date
df_max = df.groupby('order_id').agg({'reality_refund_date': 'max'})
df_max.loc[:, 'reality_refund_date'] = df_max.reality_refund_date.dt.date
df_max.rename(columns={'reality_refund_date': '实付日期_max'}, inplace=True)
df = df.merge(df_max, on='order_id', how='left')

In [8]:
df_new = df.merge(df2, on='order_id', how='left')
df_new.loc[:, '买断期数'] = np.where(df_new.支付日期==df_new.实付日期, df_new.sort, np.where(df_new.实付日期_max==df_new.实付日期, df_new.sort, 0))
# df2[df2.order_id==85952]

In [9]:
df_new = df_new[df_new.买断期数>0]
df_new = df_new.sort_values('买断期数').groupby('order_id').head(1)
df_new[['order_id', '月份', 'sort', '实付日期', '实付日期_max', '支付日期', '买断期数']]

,order_id,月份,sort,实付日期,实付日期_max,支付日期,买断期数
16131,539315.0,2023-08,1.0,2023-08-13,2023-08-13,2023-09-20,1.0
25415,797194.0,2023-11,2.0,2023-12-03,2023-12-03,2023-12-03,2.0
33243,1318577.0,2024-08,2.0,2024-09-09,2024-09-09,2024-09-09,2.0
34407,1846119.0,2024-12,2.0,2025-01-04,2025-01-04,2025-01-04,2.0
33303,1329827.0,2024-08,2.0,2024-09-02,2024-09-02,2024-09-02,2.0
...,...,...,...,...,...,...,...
9434,292146.0,2023-05,21.0,2024-12-16,2024-12-16,2024-12-16,21.0
9908,306158.0,2023-05,21.0,2025-01-06,2025-01-06,2025-01-06,21.0
9770,302283.0,2023-05,21.0,2024-12-15,2024-12-15,2024-12-15,21.0
10676,338196.0,2023-05,21.0,2024-12-17,2024-12-17,2024-12-17,21.0


In [10]:
pd.pivot_table(df_new, values='买断期数', index='月份', columns='sort', aggfunc='count')#.to_excel('F:/需求/七月需求/买断期数汇总.xlsx')

sort,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0
月份,,,,,,,,,,,,,,,,,,,,,
2023-01,NaN,2.0,1.0,NaN,1.0,NaN,3.0,1.0,3.0,10.0,30.0,9.0,3.0,5.0,2.0,3.0,13.0,20.0,NaN,NaN,NaN
2023-02,NaN,1.0,NaN,1.0,2.0,4.0,9.0,5.0,10.0,19.0,54.0,10.0,2.0,1.0,1.0,1.0,26.0,15.0,NaN,NaN,NaN
2023-03,NaN,NaN,1.0,1.0,2.0,NaN,2.0,5.0,9.0,15.0,39.0,19.0,7.0,3.0,1.0,4.0,15.0,13.0,NaN,NaN,NaN
2023-04,NaN,1.0,2.0,NaN,1.0,1.0,3.0,4.0,5.0,26.0,101.0,46.0,8.0,4.0,NaN,6.0,29.0,17.0,1.0,NaN,1.0
2023-05,NaN,NaN,3.0,2.0,NaN,NaN,1.0,3.0,7.0,7.0,57.0,48.0,4.0,NaN,2.0,1.0,4.0,3.0,4.0,3.0,4.0
2023-06,NaN,NaN,1.0,NaN,NaN,6.0,2.0,4.0,9.0,6.0,42.0,35.0,NaN,NaN,1.0,NaN,2.0,1.0,4.0,2.0,NaN
2023-07,NaN,2.0,3.0,2.0,2.0,4.0,4.0,10.0,8.0,33.0,54.0,31.0,1.0,3.0,NaN,NaN,4.0,6.0,3.0,NaN,1.0
2023-08,1.0,2.0,2.0,1.0,2.0,4.0,5.0,5.0,13.0,38.0,49.0,26.0,2.0,4.0,1.0,2.0,2.0,4.0,1.0,NaN,NaN
2023-09,NaN,NaN,2.0,3.0,6.0,4.0,5.0,3.0,9.0,35.0,41.0,46.0,5.0,3.0,3.0,1.0,1.0,NaN,1.0,NaN,NaN
